# pip installs and imports

In [ ]:
!pip install -r requirements.txt

In [ ]:
import json
import os
import time
import requests

In [ ]:
file_path = '../keys.json'
target_key = 'meaningcloud'
with open(file_path, 'r') as file:
    data = json.load(file)
key = data.get(target_key)
if key is not None:
    print(f"API KEY found")

# Calling the meaningcloud API to detect topics

In [ ]:
url = "https://api.meaningcloud.com/topics-2.0"

folderpath = r"./txt_conversations"

filepaths = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]
all_files = []

for path in filepaths:
    print(path)
    try:
        with open(path, 'r') as f:
            file = f.read()
            all_files.append(file)
    except:
        print("EXCEPTION" + path)
        continue

all_topics = []
for file in all_files:
    payload = {
        'key': key, # mine might still work
        'txt': file,
        'lang': 'en',  # 2-letter code, like en es fr ...
        'tt': 'c'  # all topics
    }
    response = requests.post(url, data=payload)
    print('Status code:', response.status_code)
    print(file)
    print(response.json())
    concepts = response.json()['concept_list']

    for concept in concepts:
        print("CONCEPT: ")
        print(f"Concept: {concept['form']}, Relevancy: {concept['relevance']}")

    all_topics.append(response.json())

    # To avoid request limit errors
    time.sleep(2.5)

    with open("./output_files/all_topics.json", mode='w', encoding='utf-8') as feedsjson3:
        json.dump(all_topics, feedsjson3)

# Cleaning the results

In [ ]:
f = open('./output_files/all_topics.json')
topics = json.load(f)
relevancy_list = []

for toic in topics:
    for concept in toic['concept_list']:
        if int(concept['relevance']) >= 50:
            print(concept['form'])
            relevancy_list.append(concept['form'])

with open("./output_files/all_relevant_topics.json", mode='w', encoding='utf-8') as feedsjson3:
    json.dump(relevancy_list, feedsjson3)
